In [2]:
import dask.distributed
import dask.dataframe as dd
import pandas as pd
import numpy as np

import urllib, json
import sklearn.neighbors

# Set up a local cluster of Dask Distributed

In [3]:
client = dask.distributed.Client()

# Read historical bike data, print head and tail

In [4]:
df = dd.read_parquet('/bigdata/citibike.parquet')

In [5]:
df.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970322,504,1 Ave & E 15 St,40.732220,-73.981659,16950,Customer,NaN,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002953,459,W 20 St & 11 Ave,40.746746,-74.007759,19816,Customer,NaN,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730286,-73.990768,237,E 11 St & 2 Ave,40.730473,-73.986725,14548,Subscriber,1980.0,2
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718941,-73.992661,499,Broadway & W 60 St,40.769154,-73.981918,16063,Customer,NaN,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734928,-73.992004,410,Suffolk St & Stanton St,40.720665,-73.985176,19213,Subscriber,1986.0,1


In [6]:
df.tail()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender
51474,441,2016-12-31 23:56:15,2017-01-01 00:03:36,284,Greenwich Ave & 8 Ave,40.739017,-74.002640,336,Sullivan St & Washington Sq,40.730476,-73.999062,16185,Subscriber,1974.0,2
51475,1026,2016-12-31 23:56:19,2017-01-01 00:13:26,281,Grand Army Plaza & Central Park S,40.764397,-73.973717,3143,5 Ave & E 78 St,40.776829,-73.963890,18267,None,1983.0,1
51476,1747,2016-12-31 23:56:35,2017-01-01 00:25:43,3424,E 106 St & Lexington Ave,40.791977,-73.945992,3295,Central Park W & W 96 St,40.791271,-73.964836,19899,Subscriber,1970.0,2
51477,951,2016-12-31 23:59:31,2017-01-01 00:15:23,3158,W 63 St & Broadway,40.771637,-73.982613,3169,Riverside Dr & W 82 St,40.787209,-73.981277,16866,Subscriber,1961.0,1
51478,1322,2016-12-31 23:59:56,2017-01-01 00:21:58,3263,Cooper Square & E 7 St,40.729237,-73.990868,498,Broadway & W 32 St,40.748550,-73.988083,25793,Subscriber,1985.0,1


In [7]:
# Count rows
df.start_station_id.compute().shape

(36902025,)

In [8]:
# Select start and end columns as separate dataframes, 
# apply range checks on lats and lons to eliminate corrupted rows

df_s = df[['start_station_id', 'start_station_name', 
           'start_station_latitude', 'start_station_longitude']]
df_e = df[['end_station_id', 'end_station_name', 
           'end_station_latitude', 'end_station_longitude']]

df_s = df_s[(df_s.start_station_latitude > 40.) & (df_s.start_station_latitude < 41.)]
df_s = df_s[(df_s.start_station_longitude + 74.0 > -0.25) & (df_s.start_station_longitude + 74.0 < 0.25)]

df_e = df_e[(df_e.end_station_latitude > 40.) & (df_e.end_station_latitude < 41.)]
df_e = df_e[(df_e.end_station_longitude + 74.0 > -0.25) & (df_e.end_station_longitude + 74.0 < 0.25)]

In [9]:
df_s.head()

,start_station_id,start_station_name,start_station_latitude,start_station_longitude
0,164,E 47 St & 2 Ave,40.753231,-73.970322
1,388,W 26 St & 10 Ave,40.749718,-74.002953
2,293,Lafayette St & E 8 St,40.730286,-73.990768
3,531,Forsyth St & Broome St,40.718941,-73.992661
4,382,University Pl & E 14 St,40.734928,-73.992004


Take the start locations, end locations, calculate mean latitude and longitude, convert to pandas, rename columns and concatenate. Sort by id. This will contain duplicates due to numerical precision and due to actual duplication due to some other reason.

In [10]:
a1 = df_s.groupby(['start_station_id', 'start_station_name']).mean().compute()
a2 = df_e.groupby(['end_station_id', 'end_station_name']).mean().compute()

a1.index = a1.index.rename(['id', 'name'])
a1.columns = ['lat', 'lon']
a2.index = a2.index.rename(['id', 'name'])
a2.columns = ['lat', 'lon']

trip_points = a1.append(a2).drop_duplicates().reset_index()
trip_points = trip_points.sort_values('id').reset_index(drop=True)

del a1, a2
trip_points.head()

,id,name,lat,lon
0,72,W 52 St & 11 Ave,40.767272,-73.993927
1,72,W 52 St & 11 Ave,40.767273,-73.993926
2,79,Franklin St & W Broadway,40.719118,-74.006668
3,79,Franklin St & W Broadway,40.719117,-74.006671
4,82,St James Pl & Pearl St,40.711174,-74.000166


In [11]:
trip_points.shape

(1413, 4)

# Read current data state from web
### This will be the authoritative list of bike stations and their coordinates

In [12]:
# Uncomment these two lines to fetch latest file and rename it

# !wget https://feeds.citibikenyc.com/stations/stations.json
# !mv stations.json stations.`date +"%Y.%m.%d.%H.%M"`.json


In [13]:
df_cur = pd.DataFrame(
    json.loads(
        open('stations.2017.04.20.09.43.json', 'r').read()
    )['stationBeanList'])
df_cur.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,20,16,,72,,2017-04-20 09:43:27 AM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,12,20,,79,,2017-04-20 09:42:54 AM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,0,0,,82,,2017-04-13 12:05:52 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,3,Not In Service,False,0
3,,35,24,,83,,2017-04-20 09:42:19 AM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,5,34,,116,,2017-04-20 09:42:20 AM,40.741776,,-74.001497,,W 17 St & 8 Ave,,W 17 St & 8 Ave,1,In Service,False,39


In [14]:
df_cur = df_cur[['id', 'stationName', 'latitude', 'longitude']].sort_values('id')
df_cur.head()

,id,stationName,latitude,longitude
0,72,W 52 St & 11 Ave,40.767272,-73.993929
1,79,Franklin St & W Broadway,40.719116,-74.006667
2,82,St James Pl & Pearl St,40.711174,-74.000165
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323
4,116,W 17 St & 8 Ave,40.741776,-74.001497


In [15]:
# We see this has less than half the rows of trip_points
print(df_cur.shape)

(664, 4)


# Do a spatial nearest neighbor search

Construct a BallTree using the actual correct coordinate data from the JSON feed. Use that to lookup the locations in trip_points.

In [16]:
bt = sklearn.neighbors.BallTree(df_cur[['latitude', 'longitude']].values, metric='euclidean')

In [17]:
np.cos(np.deg2rad(40.7))

0.75813433619765225

25 meters is approximately 0.000225˚ (not accounting for latitude)
latitude will give a factor of  cos(41˚N)=0.76 at 41˚N. So we are really testing
if the point lies within an ellipse with a radius of 25.6 meters in latitude
and 25.6*0.75=19.2 meters in longitude. Should be good enough.

In [18]:
0.000225*111194.9 # 111.1949 km per degree latitude 

25.018852499999998

In [19]:
query_results = bt.query_radius(trip_points.ix[:, 2:], 0.000225)

The value of each row indicates the row in df_cur that matches each row in trip_points
A zero for the first and second rows indicates that those rows in trip points match with
the zeroth row in df_cur. 

In [20]:
results_list = [[int(y) for y in x.tolist()] for x in query_results]

print(len(results_list))
results_list

1413


[[0],
 [0],
 [1],
 [1],
 [2],
 [2],
 [3],
 [3],
 [4],
 [4],
 [5],
 [5],
 [6],
 [6],
 [7],
 [7],
 [8],
 [8],
 [9],
 [9],
 [10],
 [10],
 [11],
 [11],
 [12],
 [12],
 [13],
 [13],
 [14],
 [14],
 [15],
 [15],
 [16],
 [16],
 [17],
 [17],
 [18],
 [18],
 [],
 [],
 [19],
 [19],
 [20],
 [20],
 [21],
 [21],
 [22],
 [22],
 [23],
 [23],
 [24],
 [24],
 [25],
 [25],
 [26],
 [26],
 [27],
 [27],
 [28],
 [28],
 [],
 [],
 [29],
 [29],
 [30],
 [30],
 [31],
 [31],
 [32],
 [32],
 [33],
 [33],
 [34],
 [34],
 [651],
 [],
 [651],
 [],
 [35],
 [35],
 [36],
 [36],
 [37],
 [37],
 [38],
 [38],
 [39],
 [39],
 [],
 [40],
 [40],
 [],
 [],
 [],
 [42],
 [42],
 [43],
 [43],
 [44],
 [44],
 [45],
 [45],
 [46],
 [46],
 [],
 [],
 [],
 [],
 [47],
 [47],
 [48],
 [48],
 [49],
 [49],
 [50],
 [50],
 [],
 [],
 [51],
 [51],
 [52],
 [52],
 [53],
 [53],
 [54],
 [54],
 [55],
 [55],
 [56],
 [56],
 [],
 [],
 [57],
 [57],
 [58],
 [58],
 [59],
 [59],
 [60],
 [60],
 [61],
 [61],
 [62],
 [62],
 [],
 [],
 [63],
 [63],
 [64],
 [64],
 [65],
 

This shows for each row in trip points, there are zero or one matches, indicating 25 meters was an appropriate choice. At 50 meters, some rows have more than one match. 

In [21]:
max((len(x) for x in results_list))

1

In [22]:
min((len(x) for x in results_list))

0

# Merge spatial query results with trip_points
Historical trip data with can be compared to the current station list.

In [23]:
# nan if no match, else row_id in df_cur
trip_points['match_id'] = np.array([np.float64(x) 
                                    if x.shape[0] > 0 else np.nan
                                    for x in query_results.T])

In [24]:
trip_points.head()

,id,name,lat,lon,match_id
0,72,W 52 St & 11 Ave,40.767272,-73.993927,0.0
1,72,W 52 St & 11 Ave,40.767273,-73.993926,0.0
2,79,Franklin St & W Broadway,40.719118,-74.006668,1.0
3,79,Franklin St & W Broadway,40.719117,-74.006671,1.0
4,82,St James Pl & Pearl St,40.711174,-74.000166,2.0


In [25]:
df_cur['id'] = df_cur['id'].astype(np.float64)

In [26]:
lookup_table = trip_points.merge(df_cur, left_on='match_id', right_index=True, how='left')
lookup_table

,id_x,name,lat,lon,match_id,id_y,stationName,latitude,longitude
0,72,W 52 St & 11 Ave,40.767272,-73.993927,0.0,72.0,W 52 St & 11 Ave,40.767272,-73.993929
1,72,W 52 St & 11 Ave,40.767273,-73.993926,0.0,72.0,W 52 St & 11 Ave,40.767272,-73.993929
2,79,Franklin St & W Broadway,40.719118,-74.006668,1.0,79.0,Franklin St & W Broadway,40.719116,-74.006667
3,79,Franklin St & W Broadway,40.719117,-74.006671,1.0,79.0,Franklin St & W Broadway,40.719116,-74.006667
4,82,St James Pl & Pearl St,40.711174,-74.000166,2.0,82.0,St James Pl & Pearl St,40.711174,-74.000165
5,82,St James Pl & Pearl St,40.711174,-74.000167,2.0,82.0,St James Pl & Pearl St,40.711174,-74.000165
6,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976325,3.0,83.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323
7,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976322,3.0,83.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323
8,116,W 17 St & 8 Ave,40.741777,-74.001498,4.0,116.0,W 17 St & 8 Ave,40.741776,-74.001497
9,116,W 17 St & 8 Ave,40.741776,-74.001493,4.0,116.0,W 17 St & 8 Ave,40.741776,-74.001497


### The above table shows rows with NaN. These stations have no current counterpart.

In [27]:
defunct_stations = lookup_table[np.isnan(lookup_table.match_id)]
defunct_stations

,id_x,name,lat,lon,match_id,id_y,stationName,latitude,longitude
38,160,E 37 St & Lexington Ave,40.748195,-73.978309,NaN,NaN,NaN,NaN,NaN
39,160,E 37 St & Lexington Ave,40.748192,-73.978310,NaN,NaN,NaN,NaN,NaN
60,218,Gallatin Pl & Livingston St,40.690286,-73.987071,NaN,NaN,NaN,NaN,NaN
61,218,Gallatin Pl & Livingston St,40.690285,-73.987069,NaN,NaN,NaN,NaN,NaN
75,233,Cadman Plaza W & Pierrepont St,40.694757,-73.990522,NaN,NaN,NaN,NaN,NaN
77,233,Cadman Plaza W & Pierrepont St,40.694754,-73.990530,NaN,NaN,NaN,NaN,NaN
88,242,Flushing Ave & Carlton Ave,40.697882,-73.973503,NaN,NaN,NaN,NaN,NaN
91,242,Flushing Ave & Carlton Ave,40.697882,-73.973503,NaN,NaN,NaN,NaN,NaN
92,243,Fulton St & Rockwell Pl,40.688151,-73.979105,NaN,NaN,NaN,NaN,NaN
93,243,Fulton St & Rockwell Pl,40.688129,-73.979028,NaN,NaN,NaN,NaN,NaN


In [28]:
defunct_stations = defunct_stations[['id_x', 'name']].drop_duplicates()
defunct_stations

,id_x,name
38,160,E 37 St & Lexington Ave
60,218,Gallatin Pl & Livingston St
75,233,Cadman Plaza W & Pierrepont St
88,242,Flushing Ave & Carlton Ave
92,243,Fulton St & Rockwell Pl
104,250,Lafayette St & Jersey St
106,250,Lafayette St & Jersey St N
116,255,NYCBS Depot - SSP
130,263,Elizabeth St & Hester St
144,271,Ashland Pl & Hanson Pl


In [29]:
defunct_trips_start = df.merge(defunct_stations, left_on=['start_station_id', 'start_station_name'],
         right_on=['id_x', 'name'])
defunct_trips_end = df.merge(defunct_stations, left_on=['end_station_id', 'end_station_name'],
         right_on=['id_x', 'name'])
defunct_trips_all = (defunct_trips_start.append(defunct_trips_end).compute().drop_duplicates())
defunct_trips_all

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id_x,name
0,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730286,-73.990768,237,E 11 St & 2 Ave,40.730473,-73.986725,14548,Subscriber,1980.0,2,293,Lafayette St & E 8 St
1,550,2013-07-01 00:01:59,2013-07-01 00:11:09,293,Lafayette St & E 8 St,40.730286,-73.990768,394,E 9 St & Avenue C,40.725212,-73.977684,16746,Customer,NaN,0,293,Lafayette St & E 8 St
2,615,2013-07-01 05:48:21,2013-07-01 05:58:36,293,Lafayette St & E 8 St,40.730286,-73.990768,507,E 25 St & 2 Ave,40.739124,-73.979736,19080,Subscriber,1984.0,2,293,Lafayette St & E 8 St
3,307,2013-07-01 06:47:47,2013-07-01 06:52:54,293,Lafayette St & E 8 St,40.730286,-73.990768,445,E 10 St & Avenue A,40.727409,-73.981422,16105,Subscriber,1973.0,1,293,Lafayette St & E 8 St
4,387,2013-07-01 07:11:46,2013-07-01 07:18:13,293,Lafayette St & E 8 St,40.730286,-73.990768,497,E 17 St & Broadway,40.737049,-73.990089,15952,Subscriber,1954.0,1,293,Lafayette St & E 8 St
5,481,2013-07-01 07:13:30,2013-07-01 07:21:31,293,Lafayette St & E 8 St,40.730286,-73.990768,469,Broadway & W 53 St,40.763439,-73.982681,16108,Subscriber,1968.0,1,293,Lafayette St & E 8 St
6,915,2013-07-01 07:42:55,2013-07-01 07:58:10,293,Lafayette St & E 8 St,40.730286,-73.990768,456,E 53 St & Madison Ave,40.759712,-73.974022,18485,Subscriber,1983.0,1,293,Lafayette St & E 8 St
7,332,2013-07-01 07:53:56,2013-07-01 07:59:28,293,Lafayette St & E 8 St,40.730286,-73.990768,380,W 4 St & 7 Ave S,40.734013,-74.002937,19540,Subscriber,1983.0,1,293,Lafayette St & E 8 St
8,687,2013-07-01 07:53:59,2013-07-01 08:05:26,293,Lafayette St & E 8 St,40.730286,-73.990768,519,Pershing Square N,40.751884,-73.977699,16270,Subscriber,1981.0,1,293,Lafayette St & E 8 St
9,472,2013-07-01 07:58:52,2013-07-01 08:06:44,293,Lafayette St & E 8 St,40.730286,-73.990768,383,Greenwich Ave & Charles St,40.735237,-74.000275,15830,Subscriber,1989.0,2,293,Lafayette St & E 8 St


# ^^
# Over five million rows that start or end at defunct stations!
## We can't drop that many given the total is 36 million!

In [56]:
defunct_start_trips_count = defunct_trips_all[['start_station_id', 'start_station_name', 'trip_duration']].groupby(['start_station_id', 'start_station_name']).count().sort_values('trip_duration', 
                                                                                               ascending=False)
bad_ids = defunct_start_trips_count[defunct_start_trips_count.trip_duration < 50].reset_index()[['start_station_id']]
defunct_start_trips_count[defunct_start_trips_count.trip_duration < 50]

,,trip_duration
start_station_id,start_station_name,
3423,West Drive & Prospect Park West,49
3329,Degraw St & Smith St,48
3414,Bergen St & Flatbush Ave,47
3339,Berkeley Pl & 6 Ave,45
3438,E 76 St & 3 Ave,45
3363,E 102 St & Park Ave,43
3404,7 St & 5 Ave,43
3419,Douglass St & 4 Ave,42
3355,E 66 St & Madison Ave,42


In [57]:
defunct_end_trips_count = defunct_trips_all[[
    'end_station_id', 'end_station_name', 'trip_duration']].groupby(
    ['end_station_id', 'end_station_name']).count().sort_values('trip_duration', ascending=False)
bad_ids_2 = (
    defunct_end_trips_count[defunct_end_trips_count.trip_duration < 50].reset_index()
)[['end_station_id']]
defunct_end_trips_count[defunct_end_trips_count.trip_duration < 50]

,,trip_duration
end_station_id,end_station_name,
3423,West Drive & Prospect Park West,49
3341,Central Park West & W 102 St,48
3414,Bergen St & Flatbush Ave,47
3283,W 89 St & Columbus Ave,44
3404,7 St & 5 Ave,43
3344,Pioneer St & Van Brunt St,39
3353,Reed St & Van Brunt St,37
3413,Wyckoff St & 3 Ave,37
3384,Smith St & 3 St,36


In [68]:
bad_ids.columns =['id']
bad_ids_2.columns = ['id']
bad_ids = bad_ids.append(bad_ids_2).drop_duplicates().sort_values('id').reset_index(drop=True)
from IPython.display import display, HTML 
display(HTML(bad_ids.T.to_html()))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85
id,3014,3017,3036,3040,3130,3166,3186,3187,3239,3240,3250,3252,3266,3283,3297,3300,3302,3304,3306,3309,3310,3313,3316,3317,3319,3320,3322,3326,3327,3328,3329,3330,3332,3333,3338,3339,3340,3341,3342,3343,3344,3345,3346,3347,3348,3351,3353,3354,3355,3356,3358,3363,3365,3366,3370,3371,3379,3381,3384,3385,3387,3390,3391,3392,3393,3394,3395,3398,3399,3401,3403,3404,3405,3407,3413,3414,3419,3421,3423,3424,3425,3432,3434,3438,3439,3440


In [73]:
bad_ids.id.T.values

array([3014, 3017, 3036, 3040, 3130, 3166, 3186, 3187, 3239, 3240, 3250,
       3252, 3266, 3283, 3297, 3300, 3302, 3304, 3306, 3309, 3310, 3313,
       3316, 3317, 3319, 3320, 3322, 3326, 3327, 3328, 3329, 3330, 3332,
       3333, 3338, 3339, 3340, 3341, 3342, 3343, 3344, 3345, 3346, 3347,
       3348, 3351, 3353, 3354, 3355, 3356, 3358, 3363, 3365, 3366, 3370,
       3371, 3379, 3381, 3384, 3385, 3387, 3390, 3391, 3392, 3393, 3394,
       3395, 3398, 3399, 3401, 3403, 3404, 3405, 3407, 3413, 3414, 3419,
       3421, 3423, 3424, 3425, 3432, 3434, 3438, 3439, 3440])

In [76]:
df2 = df.merge(bad_ids, left_on='start_station_id', right_on='id', how='left').merge(
    bad_ids, left_on='end_station_id', right_on='id', how='left'
)

In [82]:
df3 = df2[df2.id_x.isnull() & df2.id_y.isnull()]

In [83]:
df3.start_station_id.compute().shape

(36589081,)

In [81]:
df.start_station_id.compute().shape

(36902025,)